# step_renderer

> Composable renderers for the Phase 2 segmentation column and shared chrome

In [ ]:
#| default_exp components.step_renderer

In [ ]:
#| export
from typing import Any, List, Optional

from fasthtml.common import Div, Button, Span

# DaisyUI components
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles, btn_colors
from cjm_fasthtml_daisyui.utilities.semantic_colors import text_dui

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, h, min_h
from cjm_fasthtml_tailwind.utilities.typography import font_size
from cjm_fasthtml_tailwind.utilities.layout import overflow
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, gap, grow, basis
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Lucide icons
from cjm_fasthtml_lucide_icons.factory import lucide_icon

# Card stack library
from cjm_fasthtml_card_stack.components.viewport import render_viewport
from cjm_fasthtml_card_stack.components.controls import render_card_count_select
from cjm_fasthtml_card_stack.components.progress import render_progress_indicator
from cjm_fasthtml_card_stack.core.models import CardStackState
from cjm_fasthtml_card_stack.core.constants import DEFAULT_VISIBLE_COUNT, DEFAULT_CARD_WIDTH
from cjm_fasthtml_card_stack.keyboard.actions import render_card_stack_action_buttons

# Token selector library
from cjm_fasthtml_token_selector.js.core import generate_token_selector_js
from cjm_fasthtml_token_selector.components.inputs import render_hidden_inputs as render_ts_hidden_inputs

# Card stack + token selector configuration (Phase 2 segmentation instance)
from cjm_transcript_segmentation.components.card_stack_config import (
    SEG_CS_CONFIG, SEG_CS_IDS, SEG_CS_BTN_IDS,
    SEG_TS_CONFIG, SEG_TS_IDS,
)

# HTML IDs (page-specific, no combined imports)
from cjm_transcript_segmentation.html_ids import SegmentationHtmlIds

# Local imports
from cjm_transcript_segmentation.models import TextSegment, SegmentationUrls
from cjm_transcript_segmentation.utils import calculate_segment_stats
from cjm_transcript_segmentation.components.segment_card import (
    create_segment_card_renderer
)
from cjm_transcript_segmentation.components.callbacks import (
    generate_seg_callbacks_script
)

## Segmentation Toolbar & Stats

Workflow-specific toolbar with undo/reset/AI split buttons and segment statistics.
Used directly by mutation handlers for OOB swaps (via `oob=True`)
and by the combined step renderer for initial chrome population.

In [ ]:
#| export
def render_toolbar(
    reset_url: str,  # URL for reset action
    ai_split_url: str,  # URL for AI split action
    undo_url: str,  # URL for undo action
    can_undo: bool,  # Whether undo is available
    visible_count: int = DEFAULT_VISIBLE_COUNT,  # Current visible card count
    is_auto_mode: bool = False,  # Whether card count is in auto-adjust mode
    oob: bool = False,  # Whether to render as OOB swap
) -> Any:  # Toolbar component
    """Render the segmentation toolbar with action buttons and card count selector."""
    return Div(
        # Left group: Undo button
        # Note: no id= here — the keyboard system owns the "sd-seg-undo-btn" ID
        # for its hidden action button. This visible button works via hx_post directly.
        Button(
            lucide_icon("undo-2", size=4, cls=str(m.r(2))),
            "Undo",
            cls=combine_classes(btn, btn_styles.ghost, btn_sizes.sm),
            disabled=None if can_undo else "disabled",
            hx_post=undo_url,
            hx_swap="none"
        ),

        # Left spacer
        Div(cls=str(grow())),

        # Center: Card count selector with label
        Div(
            Span(
                "Cards:",
                cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70), m.r(2))
            ),
            render_card_count_select(
                SEG_CS_CONFIG, SEG_CS_IDS,
                current_count=visible_count,
                is_auto_mode=is_auto_mode,
            ),
            cls=combine_classes(flex_display, items.center)
        ),

        # Right spacer
        Div(cls=str(grow())),

        # Right group: Reset and AI Split buttons
        Div(
            Button(
                lucide_icon("rotate-ccw", size=4, cls=str(m.r(2))),
                "Reset",
                id=SegmentationHtmlIds.SEG_RESET_BTN,
                cls=combine_classes(btn, btn_styles.ghost, btn_sizes.sm),
                hx_post=reset_url,
                hx_swap="none"
            ),
            Button(
                lucide_icon("sparkles", size=4, cls=str(m.r(2))),
                "AI Split",
                id=SegmentationHtmlIds.SEG_AI_SPLIT_BTN,
                cls=combine_classes(btn, btn_colors.secondary, btn_sizes.sm),
                hx_post=ai_split_url,
                hx_swap="none"
            ),
            cls=combine_classes(flex_display, gap(2))
        ),

        id=SegmentationHtmlIds.SEG_TOOLBAR,
        cls=combine_classes(
            flex_display, gap(2), items.center
        ),
        hx_swap_oob="true" if oob else None
    )

In [ ]:
#| export
def render_seg_stats(
    segments: List[TextSegment],  # Current segments
    oob: bool = False,  # Whether to render as OOB swap
) -> Any:  # Statistics component
    """Render segmentation statistics."""
    stats = calculate_segment_stats(segments)

    return Div(
        Span(
            f"{stats['total_segments']} segments · {stats['total_words']:,} words",
            cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70))
        ),
        id=SegmentationHtmlIds.SEG_STATS,
        hx_swap_oob="true" if oob else None
    )

## Column Body Renderer

Renders the segmentation column content area with viewport and keyboard infrastructure.
Called by the combined step renderer (initial page load) and the init handler (HTMX swap).

In [ ]:
#| export
def render_seg_column_body(
    segments:List[TextSegment],  # Segments to display
    focused_index:int,  # Currently focused segment index
    visible_count:int,  # Number of visible cards in viewport
    card_width:int,  # Card stack width in rem
    urls:SegmentationUrls,  # URL bundle for all segmentation routes
    kb_system:Optional[Any]=None,  # Rendered keyboard system (None when KB managed externally)
) -> Any:  # Div with id=COLUMN_CONTENT containing viewport + infrastructure
    """Render the segmentation column content area with card stack viewport."""
    # Create card renderer callback
    card_renderer = create_segment_card_renderer(
        split_url=urls.split,
        merge_url=urls.merge,
        enter_split_url=urls.enter_split,
        exit_split_url=urls.exit_split,
    )

    # Build CardStackState for library viewport
    cs_state = CardStackState(
        focused_index=focused_index,
        visible_count=visible_count,
        card_width=card_width,
    )

    # Render viewport from library (includes focused index hidden input)
    viewport = render_viewport(
        card_items=segments,
        state=cs_state,
        config=SEG_CS_CONFIG,
        ids=SEG_CS_IDS,
        urls=urls.card_stack,
        render_card=card_renderer,
        form_input_name="segment_index",
    )

    # Generate JS: library card stack JS + focus change callback.
    # container_id is the column CONTENT area, which is the immediate parent
    # of the card stack. The column header is accounted for by containerTop
    # (content area starts below the header). This ensures the algorithm only
    # measures actual siblings of the card stack.
    callbacks_script = generate_seg_callbacks_script(
        ids=SEG_CS_IDS,
        button_ids=SEG_CS_BTN_IDS,
        config=SEG_CS_CONFIG,
        urls=urls.card_stack,
        container_id=SegmentationHtmlIds.COLUMN_CONTENT,
        focus_input_id=SEG_CS_IDS.focused_index_input,
    )

    # Token selector JS (caret navigation, display, key repeat)
    ts_script = generate_token_selector_js(SEG_TS_CONFIG, SEG_TS_IDS)

    # Keyboard system elements (optional — may be managed at combined-step level)
    kb_elements = ()
    if kb_system is not None:
        kb_elements = (kb_system.script, kb_system.hidden_inputs, kb_system.action_buttons)

    return Div(
        # Card stack viewport
        viewport,

        # Keyboard navigation system (when provided)
        *kb_elements,

        # Hidden action buttons for JS-triggered card stack nav
        render_card_stack_action_buttons(SEG_CS_BTN_IDS, urls.card_stack, SEG_CS_IDS),

        # Token selector hidden inputs (anchor + focus)
        *render_ts_hidden_inputs(SEG_TS_IDS),

        # JavaScript callbacks (library card stack JS + focus change)
        callbacks_script,

        # Token selector JS (caret navigation, display, key repeat)
        ts_script,

        id=SegmentationHtmlIds.COLUMN_CONTENT,
        cls=combine_classes(grow(), min_h(0), overflow.hidden, flex_display, flex_direction.col)
    )

## Shared Chrome Content

Functions that render segmentation-specific content for the shared chrome containers.
Used by the combined step renderer for initial page load and by the init handler for OOB swaps.

In [ ]:
#| export
def render_seg_footer_content(
    segments:List[TextSegment],  # Current segments
    focused_index:int,  # Currently focused segment index
) -> Any:  # Footer content with progress indicator and stats
    """Render footer content with progress indicator and segment statistics."""
    total_segments = len(segments)

    return Div(
        render_progress_indicator(focused_index, total_segments, SEG_CS_IDS, label="Segment"),
        render_seg_stats(segments),
        id=SegmentationHtmlIds.SEG_FOOTER,
        cls=combine_classes(flex_display, justify.between, items.center, gap(4))
    )

In [ ]:
#| export
def render_seg_mini_stats_text(
    segments:List[TextSegment],  # Current segments
) -> str:  # Compact stats string for column header badge
    """Generate compact stats string for the segmentation column header badge."""
    stats = calculate_segment_stats(segments)
    return f"{stats['total_segments']} segs \u00b7 {stats['total_words']:,} words"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()